In [1]:
!pip install segmentation-models-pytorch --quiet
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from sklearn.metrics import accuracy_score
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.9 MB/s eta 0:00:00


In [2]:
# Install required packages (if not already installed)
!pip install torch torchvision torchaudio segmentation-models-pytorch albumentations --quiet

# Import libraries
import os
import cv2
import random
import numpy as np
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
import torchvision.models as models

class FractureClassifier(nn.Module):
  def __init__(self, num_classes=8):
      super(FractureClassifier, self).__init__()
      self.backbone = models.efficientnet_b1(pretrained=True)  # Or models.efficientnet_b2

      # Get the number of input features for the classifier
      in_features = self.backbone.classifier[1].in_features

      # Replace the last layer of the classifier
      self.backbone.classifier[1] = nn.Linear(in_features, num_classes)

  def forward(self, x):
      return self.backbone(x)

In [4]:
class FractureSegmentation(nn.Module):
    def __init__(self, num_classes=7):  # No mask needed for class 0 (No Fracture)
        super(FractureSegmentation, self).__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=num_classes)

    def forward(self, x, class_label):
        mask_pred = self.unet(x)  # Shape: (batch_size, 7, H, W)

        # Extract the mask only for the predicted class
        batch_size, _, height, width = mask_pred.shape
        mask_out = torch.zeros((batch_size, 1, height, width), device=x.device)

        for i in range(batch_size):
            if class_label[i] > 0:  # Ignore No-Fracture class
                mask_out[i] = mask_pred[i, class_label[i] - 1].unsqueeze(0)

        return mask_out[:, 0] # Removing the unnecessary channel dimension

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
classifier = FractureClassifier()
segmentation = FractureSegmentation()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 64.9MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab sec

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [7]:
# Load saved weights
classifier.load_state_dict(torch.load('/content/drive/MyDrive/DL_project/classifier_model.pth', map_location=device))
segmentation.load_state_dict(torch.load('/content/drive/MyDrive/DL_project/segmentation_model.pth', map_location=device))

# Set to eval mode
classifier.eval()
segmentation.eval()


FractureSegmentation(
  (unet): Unet(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (b

In [8]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.7 MB/s eta 0:00:00


In [10]:
import gradio as gr
from PIL import Image
import torchvision.transforms as T
import numpy as np
import torch
import torch.nn as nn
import cv2
from ultralytics import YOLO

# Load models (assuming classifier, segmentation already loaded)
yolo_model = YOLO('/content/drive/MyDrive/DL_project/best_model.pt')

# ImageNet normalization values
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

# Class labels mapping
class_labels = {
    0: 'No fracture',
    1: 'elbow positive',
    2: 'fingers positive',
    3: 'forearm fracture',
    4: 'humerus fracture',
    5: 'humerus',
    6: 'shoulder fracture',
    7: 'wrist positive'
}

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean, std)
])

def denormalize(img_tensor):
    img = img_tensor.permute(1, 2, 0).numpy()
    img = img * std + mean
    img = img.clip(0, 1)
    return img

def predict(image):
    # Convert PIL to tensor
    image_pil = image.convert("RGB")
    image_tensor = transform(image_pil).unsqueeze(0).to(device)

    # Classifier prediction
    with torch.no_grad():
        class_logits = classifier(image_tensor)
        class_pred = torch.argmax(class_logits, dim=1).item()

    class_name = class_labels.get(class_pred, "Unknown")

    # Segmentation prediction
    with torch.no_grad():
        mask_pred = segmentation(image_tensor, torch.tensor([class_pred]).to(device))

    # Denormalized image for overlay
    denorm_img = denormalize(image_tensor.squeeze().cpu())

    # YOLO prediction
    yolo_results = yolo_model.predict(image_pil)
    yolo_annotated = yolo_results[0].plot()  # BGR image
    yolo_annotated = cv2.cvtColor(yolo_annotated, cv2.COLOR_BGR2RGB)

    # Convert segmentation mask to overlay
    mask_np = mask_pred.squeeze().cpu().numpy()
    mask_resized = cv2.resize(mask_np, (denorm_img.shape[1], denorm_img.shape[0]))

    overlay_img = denorm_img.copy()
    overlay_img[mask_resized > 0.5] = [1, 0, 0]  # Red mask overlay

    # Convert numpy images to PIL for display
    overlay_pil = Image.fromarray((overlay_img * 255).astype(np.uint8))
    yolo_pil = Image.fromarray(yolo_annotated)

    # Return images and classification result
    return f"Predicted Class: {class_name}", yolo_pil, overlay_pil

# Gradio interface
gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Classification Output"),
        gr.Image(label="YOLO Detection"),
        gr.Image(label="Segmentation Overlay")
    ],
    title="Fracture Detection & Segmentation"
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98ddd8dfdba02f8f6e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
